<a href="https://colab.research.google.com/github/cardstdani/practica-java/blob/main/Pr%C3%A1cticaJAVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt update
!apt install enchant --fix-missing
!apt install -qq enchant
!pip install pyenchant
!sudo apt install myspell-es
!pip install unidecode

In [19]:
import enchant
d = enchant.Dict("es")
print(d.check("habra"))
print(d.suggest("habra"))

False
['habrá', 'harba', 'habar', 'haba', 'abra', 'habría', 'hebra', 'habla', 'labra', 'cabra', 'había', 'habré']


In [56]:
import itertools
import unidecode

d = enchant.Dict("es")
def checkString(n):
  return True if d.check(n) else False

l = [chr(i).upper() for i in range(97, 123)]+[i.upper() for i in ["á", "é", "í", "ó", "ú", "ü"]]
s = sorted(["".join(i) for i in itertools.product(l, repeat=5)])

print(l)
print(s[:10], len(s))

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'Á', 'É', 'Í', 'Ó', 'Ú', 'Ü']
['AAAAA', 'AAAAB', 'AAAAC', 'AAAAD', 'AAAAE', 'AAAAF', 'AAAAG', 'AAAAH', 'AAAAI', 'AAAAJ'] 33554432


In [60]:
out = []
for i in s:
  if checkString(i):
    out.append(i)

In [61]:
print(out[100], len(out))

ACEPE 9416


In [62]:
textFile = open("Output.txt", "w")
textFile.write(" ".join(out))
textFile.close()